In [1]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout

In [2]:
import numpy as np

responses = np.load('processed_data/responses.npz')['arr_0']
responses_log = np.log(responses+1)

In [3]:
def define_discriminator(in_shape=(50, 40, 1)):
    model = Sequential()
    # downsample
    model.add(Conv2D(128, (4,4), strides=(2,2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (6,5), strides=(2,2), padding='valid'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.summary()
    return model

In [4]:
from tensorflow.keras.regularizers import L1L2
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 10 * 8
    model.add(Dense(n_nodes, input_dim=latent_dim, activity_regularizer=L1L2()))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((10, 8, 128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (7,6), strides=(2,2), padding='valid'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # generate
    model.add(Conv2D(1, (10,8), activation='tanh', padding='same'))
    model.summary()
    return model

In [5]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [6]:
def load_real_samples():
    return expand_dims(responses_log, axis=-1)

def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # select images
    X = dataset[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return X, y

def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n_samples, 1))
    return X, y

In [7]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=36):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    # save the generator model
        g_model.save('generator.h5')
    return g_model, d_loss1, d_loss2, g_loss

In [8]:
latent_dim = 200
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
# train model
g_model, d_loss1, d_loss2, g_loss = train(generator, discriminator, gan_model, dataset, latent_dim)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 20, 128)       2176      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 25, 20, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 8, 128)        491648    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 10, 8, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10240)             0         
_________________________________________________________________
dropout (Dropout)            (None, 10240)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1

KeyboardInterrupt: 

In [ ]:
plt.plot(g_loss, c='r')
plt.plot(d_loss1, c='g')
plt.plot(d_loss2, c='b');

In [ ]:
# example of loading the generator model and generating images
from tensorflow.keras.models import load_model
from numpy.random import randn
import matplotlib.pyplot as plt

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# create and save a plot of generated images (reversed grayscale)
def show_plot(examples, n):
    # plot images
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(examples[i, :, :, 0]) #cmap='gray_r')
    plt.show()

# load model
model = load_model('generator.h5')
# generate images
latent_points = generate_latent_points(200, 16)
# generate images
X = model.predict(latent_points)
# plot the result
plt.figure(figsize=(16,9))
show_plot(X, 4)

In [ ]:
plt.figure(figsize=(16,9))
plt.subplot(1, 2, 1)
plt.imshow(np.exp(model.predict(latent_points)[0].reshape(50, 40)))
plt.subplot(1, 2, 2)
plt.imshow(responses_log[0])